In [2]:
from BuildResponse import *
import ROOT as rt

pre = 'RAA_58.12.70_45.15.60__'
post= '_TR12.0_1000.0_CRnone_rhoAmin0.0_SIG4.0'
iter='iter4'
rebins = np.array([15, 20, 25, 30, 35, 40, 45],dtype='d')

def get_raa(tag, name, fout, style, color, alpha, marker_alpha=None):

    file = rt.TFile(f'{pre}{tag}{post}/RAA_{iter}.root')
    quench = file.Get('truth_AuAu')

    hg_2 = file.Get('corr_AuAu_ruu_2')
    hg_4 = file.Get('corr_AuAu_ruu_4')
    hg_6 = file.Get('corr_AuAu_ruu_6')
    truth = file.Get('truth_pp')


    fout.cd()

    hg_2 = hg_2.Rebin(len(rebins)-1, f'{name}_2', rebins)
    hg_4 = hg_4.Rebin(len(rebins)-1, f'{name}_4', rebins)
    hg_6 = hg_6.Rebin(len(rebins)-1, f'{name}_6', rebins)
    truth = truth.Rebin(len(rebins)-1, f'{name}_truth', rebins)
    quench = quench.Rebin(len(rebins)-1, f'{name}_quench', rebins)




    # hg_2.Scale(1./hg_2.GetBinContent(1))
    # hg_4.Scale(1./hg_4.GetBinContent(1))
    # hg_6.Scale(1./hg_6.GetBinContent(1))
    # truth.Scale(1./truth.GetBinContent(1))

    rat_2 = single_bin_mult(hg_2, truth, div=1)
    rat_4 = single_bin_mult(hg_4, truth, div=1)
    rat_6 = single_bin_mult(hg_6, truth, div=1)

    real = single_bin_mult(quench, truth, div=1)
    print(f'pp {truth.Integral()} AuAu {quench.Integral()} real {real.Integral()}')
    # real = single_bin_mult(real, hg_4, div=1)

    print(f'Ratio errors in {name}')
    for i in range(1, real.GetNbinsX()+1):
        x0 = real.GetBinLowEdge(i)
        x1 = x0 + real.GetBinWidth(i)
        v4 = rat_4.GetBinContent(i)
        r4 = real.GetBinContent(i)
        # print(f'{i} {v4} {r4} {(v4-r4)/r4}')
        e4 = (v4-r4)/r4*100 if r4 > 0 else 'DIV ZERO'
        print(f'Bin {i} at {x0}-{x1} has error {e4}%')
        # real.SetBinError(i, 0.)

    hg_err = make_err(rat_4, [rat_2, rat_6], color, alpha)

    rat_4.SetMarkerStyle(style)
    rat_4.SetMarkerColor(color)
    rat_4.SetLineColor(color)
    hg_err.SetFillColorAlpha(color, alpha)
    return rat_4, hg_err

def get_raa_norebin(tag, name, fout, style, color, alpha,  marker_alpha=None):
    file = rt.TFile(f'{pre}{tag}{post}/RAA_iter4.root')
    hg_6 = file.Get('rat_corr_AuAu_ruu6')
    hg_4 = file.Get('rat_corr_AuAu_ruu4')
    hg_err_4 = file.Get('err_rat_bayes_AuAu')

    fout.cd()
    hg = hg_4.Clone(name)
    hg_err = hg_err_4.Clone(f'{name}_err')

    hg.SetMarkerStyle(style)
    hg.SetMarkerColor(color)
    hg.SetLineColor(color)
    hg_err.SetFillColorAlpha(color, alpha)
    return hg, hg_err

# plot all the histograms locall
fout = rt.TFile('RAA_all.root','recreate')


# pre = pre
# post = post


hg_all_reco, hg_all_err = get_raa('reco_all',  'hg_reco_all', fout, rt.kOpenCircle, rt.kGreen+2, 0.2)
hg_cpt, hg_cpt_err = get_raa('reco_cpt',  'hg_cpt', fout, rt.kOpenStar, rt.kOrange+4, 0.2)
hg_cpt.SetMarkerSize(1.2)
hg_ang, hg_ang_err = get_raa('angularity',  'hg_ang', fout, rt.kOpenTriangleUp, rt.kBlue, 0.2)
hg_ncons, hg_ncons_err = get_raa('nconsts',  'nconsts', fout, rt.kOpenTriangleDown, rt.kCyan+1, 0.2)
hg_rhoAonly, hg_rhoAonly_err = get_raa( 'rhoAonly',  'hg_rhoAonly', fout, rt.kOpenCross, rt.kMagenta, 0.2)
hg_AB, hg_AB_err = get_raa( 'AB_method',  'AB', fout, rt.kOpenDiamond, rt.kGray+2, 0.2)
hg_AB.SetMarkerSize(1.3)

canv = rt.TCanvas("c1","c1",400,300)
canv.SetLeftMargin(0.19)
canv.SetBottomMargin(0.19)
canv.cd()


ftruth = rt.TFile(f'{pre}AB_method{post}/RAA_{iter}.root')
hg_pp = ftruth.Get("truth_pp")
hg_pp = hg_pp.Rebin(len(rebins)-1,'pp_rebin', rebins)
hg_AuAu = ftruth.Get("truth_AuAu")
hg_AuAu = hg_AuAu.Rebin(len(rebins)-1,'AuAu_rebin', rebins)
hg_truth = single_bin_mult(hg_AuAu, hg_pp, div=1)
hg_truth.SetMarkerStyle(rt.kOpenSquare)
hg_truth.SetMarkerColor(rt.kRed)
hg_truth.SetLineColor(rt.kRed)
# hg_truth.SetMarkerSize(0.06)
hg_truth.SetStats(0)
hg_truth.SetTitle("")
hg_truth.GetYaxis().SetRangeUser(0., 1.)
hg_truth.GetXaxis().SetRangeUser(15.,48.)

hg_truth.GetXaxis().SetLabelSize(0.07)
hg_truth.GetXaxis().SetTitleSize(0.07)
hg_truth.GetXaxis().SetTitle("#it{p}_{T}^{jet} [GeV/#it{c}]")

hg_truth.GetYaxis().SetLabelSize(0.07)
hg_truth.GetYaxis().SetTitleSize(0.075)
hg_truth.GetYaxis().SetTitle("R^{LeadJet}_{AA}")

hg_truth.GetXaxis().SetRangeUser(15.,45.)
hg_truth.GetYaxis().SetLabelOffset(0.02)
hg_truth.GetXaxis().SetLabelOffset(0.02)
hg_truth.GetXaxis().SetTitleOffset(1.15)
hg_truth.Draw("PE")

hg_all_reco.Draw("PE sameE")
hg_all_err.Draw("E3 same")

hg_cpt.Draw("PE same")
hg_cpt_err.Draw("E3 same")

hg_ang.Draw("PE same")
hg_ang_err.Draw("E3 same")

hg_ncons.Draw("PE same")
hg_ncons_err.Draw("E3 same")

hg_rhoAonly.Draw("PE same")
hg_rhoAonly_err.Draw("E3 same")

hg_AB.Draw("PE same")
hg_AB_err.Draw("E3 same")

ltop = 0.89
leg0 = rt.TLegend(0.2,0.75,0.5,ltop)
leg1 = rt.TLegend(0.5,0.7,0.8,ltop)
leg0.AddEntry(hg_truth, r"JETSCAPE #it{pp}", "p")
leg0.AddEntry(hg_all_reco, "NN_{AllReco}", "p")
leg0.AddEntry(hg_cpt, "NN_{pTcons}", "p")
leg0.SetLineColorAlpha(rt.kWhite, 0.)
leg0.SetFillColorAlpha(rt.kWhite, 0.)
leg1.SetFillColorAlpha(rt.kWhite, 0.)
leg0.Draw()

# leg1.AddEntry(hg_truth, "Quenched: Truth", "p")
leg1.AddEntry(hg_AB, "AB Method", "p")
leg1.AddEntry(hg_rhoAonly, "NN_{AB}", "p")
leg1.AddEntry(hg_ang, "NN_{Ang}", "p")
leg1.AddEntry(hg_ncons, "NN_{Ncons}", "p")
leg1.SetLineColorAlpha(rt.kWhite, 0.)
leg1.Draw()



# canv.SaveAs(f"Summary_{pre}_{post}_{iter}.png")
canv.SaveAs(f"Summary_{pre}_{post}_{iter}.pdf")


pp 0.00025610826877208034 AuAu 0.00017625674894251723 real 4.147556328220075
Ratio errors in hg_reco_all
Bin 1 at 15.0-20.0 has error -21.87203540862581%
Bin 2 at 20.0-25.0 has error -29.739111702681654%
Bin 3 at 25.0-30.0 has error -29.884357060269174%
Bin 4 at 30.0-35.0 has error -28.58311643286508%
Bin 5 at 35.0-40.0 has error -27.15110669043276%
Bin 6 at 40.0-45.0 has error -28.19978279188577%
pp 0.00025610826877208034 AuAu 0.00017625674894251723 real 4.147556328220075
Ratio errors in hg_cpt
Bin 1 at 15.0-20.0 has error -21.68738999545206%
Bin 2 at 20.0-25.0 has error -30.741812575136407%
Bin 3 at 25.0-30.0 has error -32.33744059400496%
Bin 4 at 30.0-35.0 has error -29.35526983465388%
Bin 5 at 35.0-40.0 has error -25.317885868139907%
Bin 6 at 40.0-45.0 has error -28.249094789646733%
pp 0.00025610826877208034 AuAu 0.00017625674894251723 real 4.147556328220075
Ratio errors in hg_ang
Bin 1 at 15.0-20.0 has error -12.658292778840377%
Bin 2 at 20.0-25.0 has error -19.816956494886792%
Bi

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file Summary_RAA_58.12.70_45.15.60____TR12.0_1000.0_CRnone_rhoAmin0.0_SIG4.0_iter4.pdf has been created
